# CropHarvest Demo

In [1]:
from cropharvest.datasets import CropHarvest

In [2]:
evaluation_datasets = CropHarvest.create_benchmark_datasets("data")

Files already downloaded.
Files already downloaded.
Files already downloaded.
Files already downloaded.
Files already downloaded.
Files already downloaded.
Files already downloaded.
Files already downloaded.


In [3]:
evaluation_datasets

[CropHarvestEval(Kenya_maize, Kenya_maize),
 CropHarvestEval(Brazil_coffee, Brazil_coffee),
 CropHarvestEval(United States of America_almond, United States of America_almond),
 CropHarvestEval(Togo_crop, togo-eval)]

In [4]:
# train a Random Forest model on the Togo dataset

In [5]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [6]:
togo_dataset = evaluation_datasets[-1]
X, y = togo_dataset.as_array(flatten_x=True)

In [7]:
X.shape, y.shape

((1290, 216), (1290,))

In [8]:
model = RandomForestClassifier()
model.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [9]:
test_preds, test_y = [], []
for _, test_instance in togo_dataset.test_data(flatten_x=True):
    
    test_preds.append(model.predict_proba(test_instance.x)[:, 1])
    test_y.append(test_instance.y)

In [10]:
print(
    f"For the Random Forest classifier, "
    f"AUC ROC score: {roc_auc_score(test_y[0], test_preds[0])}, "
)

For the Random Forest classifier, AUC ROC score: 0.8923584905660378, 
